In [7]:
!pip  install pyLDAvis

     |████████████████████████████████| 1.6 MB 1.4 MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=c2ef0eed36fc117e6b353c3a6112894507b8fe3930a0f53fb05fb2331b301db7
  Stored in directory: /Users/tatanagnedina/Library/Caches/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


In [8]:
!pip install nltk

In [9]:
!pip install spacy

In [1]:
import pandas as pd

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
import nltk; nltk.download('stopwords')


!python3 -m spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tatanagnedina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/tatanagnedina/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/tatanagnedina/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [6]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [7]:
#tokenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [12]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/Users/tatanagnedina/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

CalledProcessError: Command '/Users/tatanagnedina/Downloads/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/hx/_zbjwtz14q96svc1pjzrrctr0000gn/T/724b83_corpus.txt --output /var/folders/hx/_zbjwtz14q96svc1pjzrrctr0000gn/T/724b83_corpus.mallet' returned non-zero exit status 1.

Маллет не запускается^

In [13]:
def count_coherence(i):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return(coherence_lda)

In [14]:
def best_amount():
    a={}
    for numb in range(5,22,4):
        print(numb)
        a[numb]=count_coherence(numb)
    max_val = max(a.values())
    final_dict = {k:v for k, v in a.items() if v == max_val}
    print(a,final_dict)
    return final_dict

In [15]:
best=best_amount()

5
9
13
17
21
{5: 0.47766354760266905, 9: 0.4947097167984585, 13: 0.45580857446479217, 17: 0.4918424526814874, 21: 0.45162683124773734} {9: 0.4947097167984585}


In [16]:
for t in best:
    if t in best.keys():
        best_numb=t

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=best_numb, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
topics = lda_model.show_topics(best_numb, num_words=50, formatted=False)

In [19]:
topics

[(0,
  [('contact', 0.049800962),
   ('door', 0.035016682),
   ('pin', 0.021651216),
   ('internal', 0.019871645),
   ('panel', 0.019395871),
   ('patient', 0.019327126),
   ('doctor', 0.0193153),
   ('excellent', 0.019124733),
   ('medicine', 0.019001598),
   ('slap', 0.01507437),
   ('acquire', 0.013723539),
   ('ticket', 0.012918752),
   ('programmer', 0.011729351),
   ('safety', 0.011608614),
   ('territory', 0.009894914),
   ('session', 0.009863203),
   ('training', 0.00982842),
   ('disappear', 0.008904291),
   ('inspection', 0.00808416),
   ('horse', 0.0076430896),
   ('closed', 0.007567941),
   ('packet', 0.006986817),
   ('ultimately', 0.0068251058),
   ('deliver', 0.006374729),
   ('physician', 0.006243176),
   ('safely', 0.0061892923),
   ('bullet', 0.0061335713),
   ('handgun', 0.005935259),
   ('omit', 0.005539456),
   ('countersteere', 0.0054608816),
   ('steer', 0.004967738),
   ('lean', 0.004423538),
   ('steering', 0.0043829056),
   ('shit', 0.004371943),
   ('nerve', 

In [20]:
dict_main={}
id=0
for text in texts:
    dict_ww={
        "0":0,
        "1":0,
        "2":0,
        "3":0,
        "4":0,
        "5":0,
        "6":0,
        "7":0,
        "8":0
    }
    for word in text:
        for g in range (0,len(topics)):
            for t in range (len(topics[g][1])):
                if word == topics[g][1][t][0]:
                    dict_ww[str(g)]+=1  
    max_v = max(dict_ww.values())
    final_dict = {n:l for n, l in dict_ww.items() if l == max_v}
    for t in final_dict:
        if t in final_dict.keys():
            nom=t
    dict_main[id]=nom
    id+=1

In [21]:
sort_top=[]
for k in sorted(dict_main.keys()):
    sort_top.append(dict_main[k])

In [22]:
inverted_dict = dict()
for key, value in dict_main.items():
    inverted_dict.setdefault(value, list()).append(key)

In [23]:
inverted_dict.keys()

dict_keys(['2', '8', '5', '1', '0', '6', '7', '4', '3'])

In [38]:
df_table = pd.DataFrame()
df_table['Текст']=data
df_table['Широкий топик']=sort_top

In [39]:
df_table

,Текст,Широкий топик
0,From: (wheres my thing) Subject: WHAT car is t...,2
1,From: (Guy Kuo) Subject: SI Clock Poll - Final...,2
2,From: (Irwin Arnstein) Subject: Re: Recommenda...,2
3,From: (Tsung-Kun Chen) Subject: ** Software fo...,8
4,From: (Don A.B. Lindbergh) Subject: Diamond SS...,2
5,From: (Robert Loper) Subject: Re: SHO and SC N...,2
6,From: (Kim Richard Man) Subject: SyQuest 44M c...,2
7,From: (Kirtley Wilson) Subject: Mirosoft Offic...,8
8,Subject: Re: Dont more innocents die without t...,5
9,From: (Jon Livesey) Subject: Re: Genocide is C...,1


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vectorizer = TfidfVectorizer()

In [40]:
lst_dict={
        "0":[],
        "1":[],
        "2":[],
        "3":[],
        "4":[],
        "5":[],
        "6":[],
        "7":[],
        "8":[]}
for i in range(0,len(df_table)):
   # print(i)
    lst_dict[str(df_table['Широкий топик'][i])].append(df_table['Текст'][i])

In [ ]:
df_table['Широкий топик'][0]

In [54]:
lst_dict['0']


['From: Subject: Mazda GLC for Sale. Lines: 5 Want to sell a 1980 MAZDA GLC for $300 or b.o. Excellent working condition. Just passed inspection. Has a sun roof. No marks on body. Contact soon or (413_549-4738. kohli ',
 'From: (Kenneth Gilbert) Subject: Re: erythromycin Organization: University of Pittsburgh Lines: 14 In article (Fiberman) writes: :Is erythromycin effective in treating pneumonia? : :-fm Not only is it effective, it is in fact the drug of choice for uncomplicated cases of community-acquired penumonia. -- =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-|-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= = Kenneth Gilbert __|__ University of Pittsburgh = = General Internal Medicine | "...dammit, not a programmer!" = =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-|-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= ',
 'From: (Dave Cooper) Subject: *FOR SALE*: H-1 500... Organization: Autodesk Inc., Sausalito CA, USA Lines: 13 1975 H-1 500 Brand new top end Chambers Clean + black! 1,500$ Paul: (510) 839-2161 Please do not 

In [59]:
def get_top_tf_idf_words(vector, feature_names, top_n):
    sorted_nzs = np.argsort(vector.data)[:-(top_n+1):-1]
    return feature_names[vector.indices[sorted_nzs]]

In [90]:
list_of_5=[]
for i in range(len(lst_dict)):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(lst_dict[str(i)])
    feature_names = np.array(vectorizer.get_feature_names())
    for j,text in enumerate(lst_dict[str(i)]):
        bb=vectors[j, :]
        top_words = get_top_tf_idf_words(bb, feature_names, 5)
        list_of_5.append(top_words)

In [94]:
df_table['топ-5']=list_of_5

In [95]:
df_table

,Текст,Широкий топик,топ-5
0,From: (wheres my thing) Subject: WHAT car is t...,2,"[mazda, glc, or, condition, just]"
1,From: (Guy Kuo) Subject: SI Clock Poll - Final...,2,"[of, in, effective, kenneth, gilbert]"
2,From: (Irwin Arnstein) Subject: Re: Recommenda...,2,"[500, dave, contact, 510, top]"
3,From: (Tsung-Kun Chen) Subject: ** Software fo...,8,"[pin, input, latch, supply, data]"
4,From: (Don A.B. Lindbergh) Subject: Diamond SS...,2,"[of, horses, the, and, class]"
5,From: (Robert Loper) Subject: Re: SHO and SC N...,2,"[countersteering, the, arent, hands, feet]"
6,From: (Kim Richard Man) Subject: SyQuest 44M c...,2,"[tasmania, of, psychology, tattam, department]"
7,From: (Kirtley Wilson) Subject: Mirosoft Offic...,8,"[belief, supernatural, jon, livesey, worship]"
8,Subject: Re: Dont more innocents die without t...,5,"[predictions, francesas, whne, hovered, france..."
9,From: (Jon Livesey) Subject: Re: Genocide is C...,1,"[blacks, disproportionately, commit, minoritie..."


In [96]:
df_table.to_csv('NLP3.csv', index=False)